# 7. Multi-instrument joint analysis

## Aim
The aim of this tutorial is to do a joint spectral analysis using H.E.S.S. and Fermi-LAT data of AGN PKS 2155-305.

In [ ]:
from gammapy.data import DataStore, EventList
from gammapy.irf import PSFMap, EDispKernelMap
from gammapy.maps import MapAxis, RegionGeom, WcsGeom, Map
from gammapy.makers import SpectrumDatasetMaker, SafeMaskMaker, ReflectedRegionsBackgroundMaker
from gammapy.datasets import SpectrumDataset, Datasets, FluxPointsDataset, MapDataset
from gammapy.catalog import CATALOG_REGISTRY
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    SkyModel,
    EBLAbsorptionNormSpectralModel, 
    ExpCutoffPowerLawSpectralModel,
    create_fermi_isotropic_diffuse_model,
    TemplateSpatialModel,
    PowerLawNormSpectralModel
)
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time
from regions import CircleSkyRegion

import matplotlib.pyplot as plt

## H.E.S.S.

Select H.E.S.S. observations for PKS 2155-304 that you can find in the Gammapy datasets in `$GAMMAPY_DATA`

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1/")

In [ ]:
target_pos = SkyCoord.from_name("PKS 2155-304")

In [ ]:
obs_time = Time([53736, 54101], format="mjd", scale="tt")

In [ ]:
obs_table = data_store.obs_table
obs_table = obs_table.select_sky_circle(target_pos, 2.5*u.deg)
obs_table = obs_table.select_time_range(obs_time)

Display the content of the `obs_table`

In [ ]:
obs_table

In [ ]:
len(obs_table)

Get the observations

In [ ]:
observations = data_store.get_observations(obs_table["OBS_ID"])

In [ ]:
print(observations)

### Data reduction to DL4

In [ ]:
on_region = CircleSkyRegion(target_pos, 0.1 * u.deg)

In [ ]:
energy_axis = MapAxis.from_energy_bounds(
    0.1, 40, nbin=10, per_decade=True, unit="TeV", name="energy"
)
energy_axis_true = MapAxis.from_energy_bounds(
    0.05, 100, nbin=20, per_decade=True, unit="TeV", name="energy_true"
)

geom = RegionGeom.create(region=on_region, axes=[energy_axis])
dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true)

dataset_maker = SpectrumDatasetMaker(
    containment_correction=True, selection=["counts", "exposure", "edisp"]
)
bkg_maker = ReflectedRegionsBackgroundMaker()
safe_mask_masker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

In [ ]:
obs_ids = observations.ids
datasets = Datasets()

for obs_id, observation in zip(obs_ids, observations):
    dataset = dataset_maker.run(dataset_empty.copy(name=str(obs_id)), observation)
    dataset_on_off = bkg_maker.run(dataset, observation)
    dataset_on_off = safe_mask_masker.run(dataset_on_off, observation)
    datasets.append(dataset_on_off)

print(datasets)

Stack the dataset using `stack_reduce`

In [ ]:
hess_dataset = datasets.stack_reduce()

## Fermi-LAT

For more details on the Fermi-LAT analysis with Gammapy see https://docs.gammapy.org/2.0/tutorials/data/fermi_lat.html

Besides the files already provided in the repository, you need to download Galactic interstellar emission model `gll_iem_v07.fits` from https://fermi.gsfc.nasa.gov/ssc/data/access/lat/BackgroundModels.html as they are large files.

Read the events using `EventList.read`. Fermi-LAT events are stored in the file "ft1_00.fits"

In [ ]:
events = EventList.read("ft1_00.fits")

In [ ]:
print(events)

In [ ]:
events.table

In [ ]:
events.peek()

create an energy axis from 0.2 to 300 GeV with 8 bins

In [ ]:
energy_axis = MapAxis.from_energy_bounds(0.2, 300.0, 8, unit="GeV")
print(energy_axis)

Create a `WcsGeom` with a width of 10 deg and 0.1 deg per bin centered on the source position

In [ ]:
geom = WcsGeom.create(
    skydir=target_pos,
    binsz=0.1,
    width=(10, 10),
    axes=[energy_axis],
)
print(geom)

Create a counts map using this geometry

In [ ]:
counts = Map.from_geom(geom)
print(counts)

Fill the counts map with the event using `Map.fill_events`.

In [ ]:
counts.fill_events(events)

Have a look at the resulting count maps:

In [ ]:
counts.plot_grid();

Read the exposure using a `Map.read`. The exposure is stored in 'bexpmap_roi_00.fits'.

In [ ]:
exposure = Map.read("bexpmap_roi_00.fits")

In [ ]:
print(exposure)

In [ ]:
energy_true = exposure.geom.axes["energy_true"]

In [ ]:
irf_geom = WcsGeom(wcs=counts.geom.wcs, npix=counts.geom.npix, axes=[energy_true])

In [ ]:
exposure = exposure.interp_to_geom(irf_geom)
print(exposure)

In [ ]:
exposure.sum_over_axes().plot(add_cbar=True)

Read the psf using `PSFMap.read`. The psf is stored in 'psf_00.fits'.

In [ ]:
psf = PSFMap.read('psf_00.fits', format='gtpsf')
print(psf)

In [ ]:
psf_kernel = psf.get_psf_kernel(
    position=irf_geom.center_skydir,
    geom=irf_geom,
    max_radius="3 deg"
)
psf_kernel.psf_kernel_map.sum_over_axes().plot(stretch='log', add_cbar=True)

Create a diagonal energy dispersion matrix using `EDispKernelMap.from_diagonal_response`.

In [ ]:
energy_axis_fermi = MapAxis.from_energy_bounds(
    0.2, 300, 10, unit="GeV"
)
energy_axis_true_fermi = MapAxis.from_energy_bounds(
    0.1, 400, 15, unit="GeV", name="energy_true"
)

In [ ]:
edisp = EDispKernelMap.from_diagonal_response(
    energy_axis_fermi,
    energy_axis_true_fermi
)
edisp.get_edisp_kernel().plot_matrix()

Create the fermi dataset by putting the counts and IRFs that we have created above into a `MapDataset`.

In [ ]:
fermi_dataset = MapDataset(
    counts=counts,
    exposure=exposure,
    edisp=edisp,
    psf=psf,
    name="fermi-lat"
)

In [ ]:
fermi_dataset

## Perform joint fit
Now that we have our two dataset, we can try to fit them together.

First, we need to put them together in a `Datasets` object.

In [ ]:
datasets = Datasets([hess_dataset, fermi_dataset])

In [ ]:
datasets

Then we also need two background model specific to Fermi:

In [ ]:
diffuse_iso = create_fermi_isotropic_diffuse_model(
    filename='iso_P8R3_SOURCE_V3_v1.txt'
)
diffuse_iso.datasets_names = fermi_dataset.name
print(diffuse_iso)

In [ ]:
template_diffuse = TemplateSpatialModel.read(
    filename="gll_iem_v07.fits",
    normalize=False
)
diffuse_iem = SkyModel(
    spectral_model=PowerLawNormSpectralModel(),
    spatial_model=template_diffuse,
    name='diffuse-iem'
)

diffuse_iem.datasets_names = fermi_dataset.name

## Fitting the data:

Try to fit the data using an `ExpCutoffPowerLawSpectralModel`.

In [ ]:
spectral_model = ExpCutoffPowerLawSpectralModel(
    index=2.0,
    amplitude=1e-12 * u.Unit("cm-2 s-1 TeV-1"),
    reference=1 * u.TeV,
    lambda_=0.1 / u.TeV,
    alpha=1,
)

In [ ]:
model = SkyModel(spectral_model=spectral_model, name="PKS 2155")

datasets.models = [model]

fit_joint = Fit()
result_joint = fit_joint.run(datasets=datasets[0])

In [ ]:
model.spectral_model

In [ ]:
spectral_model.plot_error(
    energy_bounds=[1 * u.GeV, 1 * u.TeV],
    sed_type="e2dnde",
    label="EXP_Cutoff_PL",
);
plt.legend()

If you fit is working, you can also refit the data accounting for the EBL by multiplying you spectral model by an EBL model.

In [ ]:
abs_model = EBLAbsorptionNormSpectralModel.read_builtin()
abs_model.redshift.value = 0.116
source_model = SkyModel(
    spectral_model=spectral_model * abs_model,
    name='PKS 2155'
)

Produce the SED and flux-points.

In [ ]:
source_model.spectral_model.plot_error(
    energy_bounds=[1 * u.GeV, 1 * u.TeV],
    sed_type="e2dnde",
    label="EXP_Cutoff_PL x EBL",
);
plt.legend()